In [38]:
!pip install google-cloud-speech pyaudio google-cloud-texttospeech



**THIS FUNCTION IS TO LOAD VECTOR STORE THAT WE CREATE BEFORE**

In [2]:
import os
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from pydub import AudioSegment
import warnings

from pydub.playback import play
warnings.filterwarnings("ignore", category=SyntaxWarning)


from dotenv import dotenv_values
config = dotenv_values(".env")
print(config['IP_OLLAMA'])
def load_models(data_path):
    if os.path.exists(data_path):
        print("------[LOG]------\n", data_path, "\n-----------------")
        vector_store = FAISS.load_local(
            data_path,
            OllamaEmbeddings(model="bge-m3", base_url=config['IP_OLLAMA']),
            allow_dangerous_deserialization=True
        )
        return vector_store
    else:
        raise FileNotFoundError(f"Data path does not exist: {data_path}")
    
retrievers = load_models("/Users/birutekno/Documents/Project/soca-ml-app/materi-gdg/vectoring/embeddings")

https://ng92lp3ynw3p3o-11434.proxy.runpod.net
------[LOG]------
 /Users/birutekno/Documents/Project/soca-ml-app/materi-gdg/vectoring/embeddings 
-----------------


**This is function for handling RAG With Gemma 2 LLM**

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain
from langchain_ollama import ChatOllama
from langchain.schema.runnable import RunnableMap, RunnableLambda

def running_rag():
    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template="""kamu adalah bot pintar menggunakan gemma2, jawab pertanyaan user hanya berdasarkan context yang diberikan. 
        selalu jawab pertanyaan user menggunakan bahasa indonesia dan beri penjelasan yang komperhensif secara singkat namun mudah di mengerti. jika kamu tidak tahu jawabannya, sebutkan saja 
        "maaf saya tidak tahu", jangan coba untuk memberikan jawaban yang tidak relevan.

        Context: {context}\n
        Question: {question}\n
        Answer:"""
    )

    ollama_model = "gemma2:latest"
    api_url = config["IP_OLLAMA"]

    llm = ChatOllama(
        model=ollama_model,
        base_url=api_url, temperature=0.3
    )

    retriever = retrievers.as_retriever(
        search_type="similarity", search_kwargs={"k": 4}
    )

    def debug_documents(inputs):
        docs = inputs.get("documents", [])
        for doc in docs:
            print(f"[DEBUG] Chunk to LLM: {doc.page_content}")
        return inputs

    pipeline = RunnableMap({
        "retriever": RunnableLambda(lambda inputs: {"documents": retriever.get_relevant_documents(inputs["question"])}),
        "debugger": RunnableLambda(debug_documents),
        "qa_chain": RetrievalQAWithSourcesChain.from_chain_type(
            llm=llm,
            retriever=retriever,
            chain_type="stuff",
            chain_type_kwargs={"prompt": prompt_template, "document_variable_name": "context"},
            return_source_documents=True,
            verbose=True
        ),
    })
    return pipeline

def invoke_rag(question):
    print("this is question", question)
    pipeline = running_rag()
    result = pipeline.invoke({
        "question": question
    })

    answer = result["qa_chain"]["answer"]
    source_list = [
        str(doc.metadata["source"]).split("/")[-1]
        for doc in result["qa_chain"]["source_documents"]
        if str(doc.metadata["source"]).split(".")[-1] not in ["jpg", "png"]
    ]
    source_list = list(set(source_list))
    print("souce list", source_list)
    return answer


In [4]:
# from google.cloud import speech_v1p1beta1 as speech
import os
from dotenv import dotenv_values
import io
config = dotenv_values(".env")
credential_keys = config['CREDENTIAL_PATH']
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=credential_keys
from google.cloud import speech_v1 as speech
from google.cloud import speech, texttospeech

import io

def transcribe_audio(audio_file_path):
    client = speech.SpeechClient()

    with io.open(audio_file_path, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="id-ID", 
    )

    try:
        response = client.recognize(config=config, audio=audio)
        print(response)
        if not response.results:
            return "No transcription results. Ensure audio clarity and configuration correctness."

        transcript = " ".join(result.alternatives[0].transcript for result in response.results)
        return transcript

    except Exception as e:
        return f"Error during transcription: {e}"



In [5]:

def post_tts(text, emotion='normal', pitch=1, gender="Female", language_code="id-ID", ssml_name="id-ID-Wavenet-B"):
    """Generates a TTS audio from the input text."""
    client = texttospeech.TextToSpeechClient()
    input_text = texttospeech.SynthesisInput(text=text)

    ssml_gender = texttospeech.SsmlVoiceGender.FEMALE if gender != "Male" else texttospeech.SsmlVoiceGender.MALE
    speaking_rate = 1
    pitch = int(pitch)
    volume_gain_db = 0
    if emotion.lower() == "excited":
        speaking_rate = 1.1
        pitch = 5.0
        volume_gain_db = 6.0

    voice = texttospeech.VoiceSelectionParams(
        language_code=language_code,
        name=ssml_name,
        ssml_gender=ssml_gender,
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3,
        speaking_rate=speaking_rate,
        pitch=pitch,
        volume_gain_db=volume_gain_db,
    )

    response = client.synthesize_speech(
        request={"input": input_text, "voice": voice, "audio_config": audio_config}
    )

    return response.audio_content

def play_audio(content,filename="response_audio.mp3"):
    """Plays audio content using pydub."""
    if not content:
        raise ValueError("Audio content is empty or invalid.")
    with open(filename, 'wb') as audio_file:
        audio_file.write(content)
    audio = AudioSegment.from_file(filename, format="mp3")
    play(audio)


In [6]:
import os
from google.cloud import speech
import pyaudio
from six.moves import queue

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_keys

RATE = 16000
CHUNK = int(RATE / 10)

class MicrophoneStream:
    def __init__(self, rate, chunk):
        self.rate = rate
        self.chunk = chunk

        self.buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self.audio_interface = pyaudio.PyAudio()
        self.audio_stream = self.audio_interface.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=self.rate,
            input=True,
            frames_per_buffer=self.chunk,
            stream_callback=self._fill_buffer,
        )
        self.closed = False
        return self

    def __exit__(self, type, value, traceback):
        self.audio_stream.stop_stream()
        self.audio_stream.close()
        self.closed = True
        self.buff.put(None)
        self.audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        self.buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            chunk = self.buff.get()
            if chunk is None:
                return
            data = [chunk]
            while True:
                try:
                    chunk = self.buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break
            yield b"".join(data)


def listen_and_transcribe():
    """Streams microphone input to Google Speech-to-Text for transcription."""
    client = speech.SpeechClient()

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code="id-ID"
    )
    streaming_config = speech.StreamingRecognitionConfig(config=config, interim_results=True)

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (speech.StreamingRecognizeRequest(audio_content=content) for content in audio_generator)

        responses = client.streaming_recognize(streaming_config, requests)

        for response in responses:
            for result in response.results:
                if result.is_final:
                    result_speech = result.alternatives[0].transcript.strip()
                    if result_speech.lower() in ["terima kasih", "terimakasih"] :
                        print("Detected keyword for stopping")
                        return  
                    answers = invoke_rag(result_speech)
                    print("answer", answers)
                    tts_response = post_tts(answers)
                    # print("this is response",tts_response)
                    play_audio(tts_response)


if __name__ == "__main__":
    print("Speak into the microphone...")
    listen_and_transcribe()


Speak into the microphone...
this is question Siapa nama Menteri Keuangan saat ini


> Entering new RetrievalQAWithSourcesChain chain...


/var/folders/m4/_5r061fs01dfpsm_ydkg8_f00000gn/T/ipykernel_61605/1267120307.py:37: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  "retriever": RunnableLambda(lambda inputs: {"documents": retriever.get_relevant_documents(inputs["question"])}),



> Finished chain.
souce list ['nama_mentri.pdf']
answer Nama Menteri Keuangan saat ini adalah Sri Mulyani Indrawati. 



I0000 00:00:1732897110.458835 1502679 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1732897110.617864 1502679 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


: 

In [17]:
import os

file_path = "/Users/birutekno/Documents/Project/soca-ml-app/materi-gdg/response_audio.mp3"
if not os.path.exists(file_path):
    print("File does not exist!")
else:
    print("File exists!")

audio = AudioSegment.from_file("/Users/birutekno/Documents/Project/soca-ml-app/materi-gdg/response_audio.mp3", format="mp3")
play(audio)


File exists!


: 